In [25]:
!pip install --upgrade boto3 -i https://pypi.tuna.tsinghua.edu.cn/simple/
!pip install --upgrade sagemaker -i https://pypi.tuna.tsinghua.edu.cn/simple/

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple/, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 102.7 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.31.7
    Uninstalling botocore-1.31.7:
      Successfully uninstalled botocore-1.31.7
  Attempting uninstall: boto3
    Found existing installation: boto3 1.28.7
    Uninstalling boto3-1.28.7:
      Successfully uninstalled boto3-1.28.7
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.27.71 requires botocore==1.29.71, but you have botocore 1.31.9 which is incompatible.
awscli 1.27.71 requires PyYAML<5.5,>=3.10, but you have pyyaml 6.0 which is incompatible.
Looking in indexes: https://pypi.tuna.tsinghua.edu.c

In [26]:
import boto3
import sagemaker

account_id = boto3.client('sts').get_caller_identity().get('Account')
region_name = boto3.session.Session().region_name

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()

print(role)
print(bucket)

if "cn-" in region_name:
    with open('./code/requirements.txt', 'r') as original: data = original.read()
    with open('./code/requirements.txt', 'w') as modified: modified.write("-i https://pypi.tuna.tsinghua.edu.cn/simple\n" + data)

!touch dummy
!tar czvf model.tar.gz dummy
assets_dir = 's3://{0}/{1}/assets/'.format(bucket, 'llm_chinese')
model_data = 's3://{0}/{1}/assets/model.tar.gz'.format(bucket, 'llm_chinese')
!aws s3 cp model.tar.gz $assets_dir
!rm -f dummy model.tar.gz

model_name = None
entry_point = 'inference.py'
framework_version = '1.13.1'
py_version = 'py39'
model_environment = {
    'SAGEMAKER_MODEL_SERVER_TIMEOUT':'600', 
    'SAGEMAKER_MODEL_SERVER_WORKERS': '1', 
}

from sagemaker.pytorch.model import PyTorchModel

model = PyTorchModel(
    name = model_name,
    model_data = model_data,
    entry_point = entry_point,
    source_dir = './code',
    role = role,
    framework_version = framework_version, 
    py_version = py_version,
    env = model_environment
)

endpoint_name = 'pytorch-inference-llm-v1'
#instance_type = 'ml.p3.2xlarge'
instance_type='ml.g4dn.2xlarge' 

instance_count = 1

from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
predictor = model.deploy(
    endpoint_name = endpoint_name,
    instance_type = instance_type, 
    initial_instance_count = instance_count,
    serializer = JSONSerializer(),
    deserializer = JSONDeserializer()
)

arn:aws-cn:iam::507392672631:role/NotebookStack-SmartSearchNotebookRole6F6BB12B-XOVHSG6091PM
sagemaker-cn-north-1-507392672631
dummy
upload: ./model.tar.gz to s3://sagemaker-cn-north-1-507392672631/llm_chinese/assets/model.tar.gz
----------------!

### 测试


In [27]:
#休眠2分钟,确保模型可以完全加载
import time
time.sleep(120)

In [28]:
inputs= {
    "ask": "你好!"

}

response = predictor.predict(inputs)
print(response["answer"])

inputs= {
    "ask": "晚上睡不着应该怎么办"

}

response = predictor.predict(inputs)
print(response["answer"])

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (0) from model with message "Your invocation timed out while waiting for a response from container model. Review the latency metrics for each container in Amazon CloudWatch, resolve the issue, and try again.". See https://cn-north-1.console.aws.amazon.com/cloudwatch/home?region=cn-north-1#logEventViewer:group=/aws/sagemaker/Endpoints/pytorch-inference-llm-v1 in account 507392672631 for more information.

In [ ]:
inputs= {
    "ask": "列出一些年夜饭好意头的菜肴以及其寓意!"

}

response = predictor.predict(inputs)
print(response["answer"])

inputs= {
    "ask": "帮我写一篇人工智能课程的教案，1000字"

}

response = predictor.predict(inputs)
print(response["answer"])

In [ ]:
inputs= {
    "ask": "怎么修改huggingface transformers的model cache位置"

}

response = predictor.predict(inputs)
print(response["answer"])

inputs= {
    "ask": "用python3写出快速排序代码"

}

response = predictor.predict(inputs)
print(response["answer"])

In [ ]:
#我们来查看一下刚部署的这个模型对应的SageMaker inference endpoint
sagemaker_endpoint_name = predictor.endpoint_name
sagemaker_endpoint_name

利用已经在SageMaker real time inference endpoint部署的LLM模型来模拟单轮对话和多轮对话。

下面的代码建议在SageMaker Notebook上来运行。

In [2]:
import json
import boto3

client = boto3.client('runtime.sagemaker')
sagemaker_endpoint_name='pytorch-inference-llm-v1'
def query_endpoint_with_json_payload(encoded_json):
    response = client.invoke_endpoint(EndpointName=sagemaker_endpoint_name, ContentType='application/json', Body=encoded_json)
    return response

def parse_response_texts(query_response):
    model_predictions = json.loads(query_response['Body'].read())
    generated_text = model_predictions["answer"]
    return generated_text


先简单测试一下针对单个问题的回答

In [3]:
payload = {"ask": """13英寸MacBook Pro。title: AI抢饭碗引发好莱坞大罢工！业内大佬警告：将产生多米诺骨牌效应
ctime: 1689554822
brief: ①前派拉蒙影业CEO巴里·迪勒表示，若不尽快解决好莱坞编剧和演员工会的罢工问题，将会带来毁灭性的影响；
②当地时间14日，美国演员工会宣布支持美国编剧协会，正式加入罢工行列，这也是60多年来好莱坞最大的两个工会组织首次同时罢工。
content: <p><strong>财联社7月17日讯（编辑 卞纯）</strong>人工智能（AI）对于美国好莱坞的冲击波仍在发酵。继美国编剧协会开始罢工以后，美国演员工会也加入了。</p>
<p>当地时间14日，美国演员工会宣布支持美国编剧协会，正式加入罢工行列，这也是60多年来好莱坞最大的两个工会组织首次同时罢工。此前，工会方面与制片公司未能就劳资问题达成协议。。</第2条信息>
<第3条信息>在人工智慧浪潮带动下，七家重量级科技股股价飙升40%至200%以上，成为标普500指数今年迄今17%的涨幅的主因，近期这几家公司更是被外界称为“绝地七骑士”，包括苹果、微软、谷歌、亚马逊、脸书、英伟达和特斯拉。</p>
<p>  目前华尔街对七巨头的财报获利预期都相当高，美银全球研究预计，未来12个月，这些公司的收益将平均增长19%，是标普500指数其他成份股8%预期增幅的2倍多。</p>
<p>在此背景下，特斯拉将于美东时间周三盘后（北京时间7月20日清晨）率先公布2023年二季度财报。值得关注的是，在最近一个月市场普遍预期Model 3将新增线控转向。</p>
<p><strong>一、自动驾驶必备黑科技，各路资本纷纷入局</strong></p>
<p>过去几年，智能汽车领域最热门的投资赛道，大多集中在智能驾驶的感知以及决策环节，比如激光雷达、毫米波雷达、AI芯片等。今年各路资本纷纷涌向线控底盘领域。。</第3条信息>
<第5条信息>今日两市炸板个股较多，炸板率维持高位，目前超40%。短线情绪指标显示，市场短线情绪午后持续回落，接近低迷区。title: 金融学子就业高度内卷：卷学历、卷证书、卷实习，仍不敌复合人才
ctime: 1689574264
brief: ①2022年金融财经专业的应届生超过100万，占高校毕业生的10%；
②人才饱和、竞争激烈将工作门槛无限拉高，金融学子们从各方面高度内卷；
③现在，复合背景的金融人才往往更受券商青睐。
content: <p><strong>财联社7月17日讯（记者 肖斐歆）</strong>金融专业曾是公认就业前景广、薪资水平高的行业，其中券商更是金融精英聚集地，是不少年轻人追逐的梦想工作去处。</p>
<p>据教育部统计，2022年高校毕业生数量达1076万人，金融财经专业的应届生超过100万。不仅如此，学霸也偏爱金融专业，高考状元扎堆去顶级学府学金融似乎已是常态。</p>
<p>门槛的无限拉高让金融学子们高度内卷，卷学历、卷证书、卷实习……伴随着券商降薪大潮的袭来和复合背景人才更受青睐的趋势，金融学子该何去何从？。</第5条信息>
<第6条信息></p>title: 金梓才获擢升财通基金副总经理，绩而优则仕与80后标签打足，年内已近10位基金经理获提升
ctime: 1689554818
brief: ①金梓才的升任可以窥见财通基金的权益布局思路；
②“绩优则仕”留人才，年内已有近10位基金经理升至副总。
content: <p><strong>财联社7月17日讯（记者 闫军）</stro
基于以上已知信息，简洁和专业的来回答用户的问题，并告知是依据哪些信息来进行回答的。
如果无法从已知信息中得到答案，请仅回答 "根据已知信息无法回答该问题" 或 "没有提供足够的相关信息"，不允许在答案中添加编造成分，答案请使用中文。
问题是:
目前华尔街对七巨头期望怎么样？"""}
query_response = query_endpoint_with_json_payload(json.dumps(payload).encode('utf-8'))
#print(query_response)
#display_answer(query_response)
generated_texts = parse_response_texts(query_response)
print(generated_texts)

根据已知信息，华尔街对七巨头的财报获利预期都相当高。美银全球研究预计，未来12个月，这些公司的收益将平均增长19%，是标普500指数其他成份股8%预期增幅的2倍多。这反映出华尔街对七巨头的未来表现持乐观态度。


In [ ]:
payload = {"ask": """已知信息:13英寸MacBook Pro。title: AI抢饭碗引发好莱坞大罢工！业内大佬警告：将产生多米诺骨牌效应
ctime: 1689554822
brief: ①前派拉蒙影业CEO巴里·迪勒表示，若不尽快解决好莱坞编剧和演员工会的罢工问题，将会带来毁灭性的影响；
②当地时间14日，美国演员工会宣布支持美国编剧协会，正式加入罢工行列，这也是60多年来好莱坞最大的两个工会组织首次同时罢工。
content: <p><strong>财联社7月17日讯（编辑 卞纯）</strong>人工智能（AI）对于美国好莱坞的冲击波仍在发酵。继美国编剧协会开始罢工以后，美国演员工会也加入了。</p>
<p>当地时间14日，美国演员工会宣布支持美国编剧协会，正式加入罢工行列，这也是60多年来好莱坞最大的两个工会组织首次同时罢工。此前，工会方面与制片公司未能就劳资问题达成协议。。</第2条信息>
<第3条信息>在人工智慧浪潮带动下，七家重量级科技股股价飙升40%至200%以上，成为标普500指数今年迄今17%的涨幅的主因，近期这几家公司更是被外界称为“绝地七骑士”，包括苹果、微软、谷歌、亚马逊、脸书、英伟达和特斯拉。</p>
<p>  目前华尔街对七巨头的财报获利预期都相当高，美银全球研究预计，未来12个月，这些公司的收益将平均增长19%，是标普500指数其他成份股8%预期增幅的2倍多。</p>

基于以上已知信息，简洁和专业的来回答用户的问题，并告知是依据哪些信息来进行回答的。
如果无法从已知信息中得到答案，请仅回答 "根据已知信息无法回答该问题" 或 "没有提供足够的相关信息"，不允许在答案中添加编造成分，答案请使用中文。
问题是:
目前华尔街对七巨头期望怎么样？"""}
query_response = query_endpoint_with_json_payload(json.dumps(payload).encode('utf-8'))
#print(query_response)
#display_answer(query_response)
generated_texts = parse_response_texts(query_response)
print(generated_texts)

In [ ]:
payload = {"ask": "信息抽取：\n2022年世界杯的冠军是阿根廷队伍，梅西是MVP\n问题：国家名，人名\n答案："}
query_response = query_endpoint_with_json_payload(json.dumps(payload).encode('utf-8'))
generated_texts = parse_response_texts(query_response)
print(generated_texts)

单轮对话：每个问题/query都是独立的，问题之间没有关联性。

In [ ]:
#1.首先需要一个简单的开场拍。
print("用户：你好！\nAI：您好!我是AI。我可以回答您的问题、写文章、写作业、翻译，对于一些法律等领域的问题我也可以给你提供信息。")

#2.在同一个session中持续对话，但是每次对话之间没有什么关联。
while True:
    command = input("用户：")
    if command == 'quit':
        break;
    
    #print(command)
    payload = {"ask": "\n用户："+ command + "\n"}
    #print(payload["ask"])
    query_response = query_endpoint_with_json_payload(json.dumps(payload).encode('utf-8'))
    generated_texts = "AI：" + parse_response_texts(query_response)
    print(generated_texts)

多轮对话模拟：我们这里来测试一下多轮对话能力。

In [ ]:
#1.首先需要开场拍来引导AI，其实就是给它一个上下文来启动所谓的对话session。
payload = {"ask": "用户：你好！\nAI：您好!我是AI。我可以回答您的问题、写文章、写作业、翻译，对于一些法律等领域的问题我也可以给你提供信息。"}
print(payload["ask"])
generated_texts = ""

#在这里简单模拟多轮对话时，发送给SageMaker endpoint的payload会越来越大，这里对payload大致做一个限制。
session_len = 10 * 1024 * 1024 

#2.在同一个session中持续对话，为了有多轮对话的效果，把每一轮的信息(问题-回答对)都带上来告诉LLM之前的上下文。
while True:
    command = input("用户：")
    if command == 'quit':
        break;
    
    #print(command)
    whole_context = payload["ask"] + generated_texts + "\n用户："+ command + "\n"
    payload = {"ask": whole_context}
    if len(whole_context) > session_len:
        print("上下文信息太多了，当前对话session要退出了！")
        break;
    #print(payload["ask"])
    query_response = query_endpoint_with_json_payload(json.dumps(payload).encode('utf-8'))
    generated_texts = "AI：" + parse_response_texts(query_response)
    print(generated_texts)

### 删除SageMaker  Endpoint
删除推理服务

In [ ]:
#predictor.delete_endpoint()